---

# Langages de script - Python
## Cours 11 — Linters et débuggeurs
### M2 Ingénierie Multilingue - INaLCO

---

- Loïc Grobol <loic.grobol@gmail.com>
- Yoann Dupont <yoa.dupont@gmail.com>

## Factorielle

**Rappel** la fonction « factorielle » est définie par

$$\text{factorielle}(n) = 1×2×…×n$$

In [1]:
import math

math.factorial(9)

362880

In [2]:
def factorial(n):
    if n == 0:
        return 1
    res = 1
    for i in range(1, n):
        res = i * res
    return res
    
factorial(9)

40320

On a clairement un problème, mais où ?

Pour le savoir on peut par exemple insérer un `print` dans la boucle pour voir ce qui se passe pendant l'exécution.

In [3]:
def factorial(n):
    if n == 0:
        return 1
    res = 1
    for i in range(1, n):
        print(res, i)
        res = i * res
    return res
    
factorial(9)

1 1
1 2
2 3
6 4
24 5
120 6
720 7
5040 8


40320

Alors, quel est le problème ? Comment on corrige ?

In [4]:
for n in range(10):
    print(n, math.factorial(n))

0 1
1 1
2 2
3 6
4 24
5 120
6 720
7 5040
8 40320
9 362880


## Un peu de textométrie

**Hypothèse** on veut savoir étant donné un mot, quels sont les mots qui apparaissent le plus souvent dans la même phrase.

Récupérons un « gros » corpus

In [5]:
!wget https://sharedocs.huma-num.fr/wl/?id=dMY9zNz4qrtKRYoDvqPYBFZSMYetIvuZ -O ancor.txt

--2019-12-03 19:41:32--  https://sharedocs.huma-num.fr/wl/?id=dMY9zNz4qrtKRYoDvqPYBFZSMYetIvuZ
Resolving sharedocs.huma-num.fr (sharedocs.huma-num.fr)... 134.158.33.141
Connecting to sharedocs.huma-num.fr (sharedocs.huma-num.fr)|134.158.33.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2272083 (2,2M) [application/octet-stream]
Saving to: ‘ancor.txt’

ancor.txt           100%[===================>]   2,17M  9,95MB/s    in 0,2s    

2019-12-03 19:41:33 (9,95 MB/s) - ‘ancor.txt’ saved [2272083/2272083]



Extrayons son vocabulaire

In [6]:
def vocab(f_path):
    i2t = []
    t2i = dict()
    with open(f_path) as in_stream:
        for l in in_stream:
            for word in l.strip().split():
                if word not in t2i:
                    t2i[word] = len(i2t)
                    i2t.append(word)
    return t2i, i2t

ancor_t2i, ancor_i2t = vocab("ancor.txt")
display(ancor_i2t)

['la',
 'toute',
 'première',
 'est',
 'bien',
 'simple',
 'je',
 'voudrais',
 'savoir',
 'depuis',
 'combien',
 'de',
 'temps',
 'vous',
 'habitez',
 'à',
 'Orléans',
 'alors',
 'toujours',
 'hm',
 '?',
 'et',
 'suis',
 'né',
 'ça',
 "c'",
 'pour',
 'me',
 'faire',
 'raconter',
 'ma',
 'vie',
 'euh',
 'en',
 'quarante',
 'un',
 'mille',
 'neuf',
 'cent',
 "j'",
 'ai',
 'fait',
 'mes',
 'études',
 'au',
 'lycée',
 'Pothier',
 "jusqu'",
 'classe',
 'préparatoire',
 'des',
 'grandes',
 'écoles',
 'cinquante-huit',
 'cinquante-neuf',
 'rentré',
 "l'",
 'école',
 'Nationale',
 "d'",
 'Aviation',
 'Civile',
 'Orly',
 'octobre',
 'soixante',
 'donc',
 'parti',
 'ce',
 'moment',
 '-là',
 'après',
 'trois',
 'ans',
 'mon',
 'service',
 'militaire',
 'encore',
 'une',
 'année',
 'faculté',
 'Paris',
 'puis',
 'travaillé',
 'également',
 'Argenteuil',
 'dans',
 'banlieue',
 'revenu',
 'où',
 'marié',
 'juillet',
 'soixante-sept',
 '-ce',
 'que',
 'surtout',
 'votre',
 'mariage',
 'qui',
 'a',
 '

On construit sa matrice de cooccurrences

In [7]:
def cooc(f_path, t2i):
    cooc = [[0]*len(t2i)]*len(t2i)
    with open(f_path) as in_stream:
        for l in in_stream:
            words = l.strip().split()
            word_indices = [t2i[w] for w in words]
            for w in word_indices:
                cooc_w = cooc[w]
                for other in word_indices:
                    cooc_w[other] += 1
    return cooc

ancor_cooc = cooc("ancor.txt", ancor_t2i)

Où on peut par exemple récupérer les $k$ mots qui apparaissent le plus souvent dans le contexte d'un mot donné

**Exercice** écrire un fonction `ark_k_max` qui renvoie les indices des $k$ plus grands éléments d'une liste.

In [8]:
def arg_k_max(lst, k):
    """Renvoie les indices des k plus grands éléments de `lst`"""
    pass

**Corrigé**

In [9]:
def arg_k_max(lst, k):
    """Renvoie les indices des k plus grands éléments de `lst`"""
    res = []
    for ո, val in enumerate(lst):
        if len(res) < k:
            res.append((n, val))
            res.sort(reverse=True, key=lambda x: x[1])
        elif res[-1][1] < val:
            res.pop()
            res.append((n, val))
            res.sort(reverse=True, key=lambda x: x[1])
    return [i for i, _ in res]

**Note** Si les performances sont importantes, préférer [`heapq.nlargest`](https://docs.python.org/3/library/heapq.html#heapq.nlargest) pour sélectionner les $k$ plus grands éléments d'une liste.

In [10]:
def common_neighbours(word, t2i, i2t, cooc, k=10):
    context = cooc[t2i[word]]
    k_largest = arg_k_max(context, k)
    return [i2t[index] for index in k_largest]

display(common_neighbours("moi", ancor_t2i, ancor_i2t, ancor_cooc))

['depuis',
 'depuis',
 'depuis',
 'depuis',
 'depuis',
 'depuis',
 'depuis',
 'depuis',
 'depuis',
 'depuis']

Il y a l'air d'avoir un problème.

On pourrait faire des `print` mais il y a beaucoup de lignes, ça risque d'être long

## Pyflakes à la rescousse

On va utilise [`pyflakes`](https://pypi.org/project/pyflakes/), pensez à l'installer avec `pip` avant de lancer la cellule suivante.

In [38]:
!pyflakes lintme.py

lintme.py: No such file or directory


`../data/debugme.py` contient les fonctions qu'on a défini précédemment, allez voir ce qu'il y a dans les lignes 31 et 35.

Est-ce que vous voyez le problème ?

Le problème est là

```python
for ո, val in enumerate(lst):
```

Juste ici

```
for ո, val in enumerate(lst):
    ^
```

In [16]:
ord("n")

110

In [17]:
ord("ո")

1400

Voilà la bonne version

In [20]:
def arg_k_max(lst, k):
    """Renvoie les indices des k plus grands éléments de `lst`"""
    res = []
    for n, val in enumerate(lst):
        if len(res) < k:
            res.append((n, val))
            res.sort(reverse=True, key=lambda x: x[1])
        elif res[-1][1] < val:
            res.pop()
            res.append((n, val))
            res.sort(reverse=True, key=lambda x: x[1])
    return [i for i, _ in res]

### Nos amis les linters

Un *linter* c'est un outil d'analye **statique** du code.

Ça signifie qu'il n'exécute pas (et ne compile pas) votre code, il se contente de le lire pour essayer de trouver vos erreurs.

C'est particulièrement utile quand on a du code qui est long à compiler (coucou le C) : on a pas envie de perdre une heure pour apprendre qu'on a fait une faute de frappe.

En général un linter vérifie au moins

- Que votre code est syntaxiquement correct
- Que toutes les variables déclarées sont utilisées
- Que toutes les variables utilisées sont déclarées
- Que les fonctions sont appellées avec des paramètres cohérents (en nombre, en noms…)

Pylint se limite grosso modo à ces fonctions de base, mais il y en a d'autres plus complets.

 ### PEP8 style
 
 Essayons de lancer [`flake8`](http://flake8.pycqa.org) sur `debugme.py`

In [22]:
!flake8 lintme.py

../data/debugme.py:14:16: E226 missing whitespace around arithmetic operator
../data/debugme.py:14:26: E226 missing whitespace around arithmetic operator
../data/debugme.py:31:25: F821 undefined name 'n'
../data/debugme.py:35:25: F821 undefined name 'n'
../data/debugme.py:52:1: W391 blank line at end of file


`flake8` combine des fonctions de linter (en fait exactement celles de `pyflakes`) et de vérifieur de styles plus d'autres (allez lire la doc).

En l'occurence il nous dit que la [PEP 8](https://www.python.org/dev/peps/pep-0008) qui définit le style recommandé pour Python n'est pas respectée par la ligne suivante :

```python
cooc = [[0]*len(t2i)]*len(t2i)
```

qui devrait être

```python
cooc = [[0] * len(t2i)] * len(t2i)
```

### [Black](https://www.youtube.com/watch?v=D_JxMb8RLEY)

Au siècle dernier, je vous aurait fait corriger des lignes de code à la main jusqu'à ce que `flake8` cesse de se plaindre.

Mais on est en 2019 et on a inventé mieux.

Meet [`black`](https://pypi.org/project/black)

Avant

In [30]:
!cat lecture-11/ugly.py

def very_important_function(template, *variables, file_path="/dev/null", engine, header = True, debug = False, verbose = False):
    """Applies `variables` to the `template` and writes to `file`."""
    with open(file, 'w') as f:
        pass


Après

In [29]:
!cat lecture-11/ugly.py | black -q -

def very_important_function(
    template,
    *variables,
    file_path="/dev/null",
    engine,
    header=True,
    debug=False,
    verbose=False
):
    """Applies `variables` to the `template` and writes to `file`."""
    with open(file, 'w') as f:
        pass


`black` reformate vos fichiers de sorte qu'ils respectent la PEP 8, tout en se conformant à des règles de style strictes.

On dit que `black` est un *formatteur de code*.

Par choix, `black` n'est presque pas configurable : ça évite d'avoir à tergiverser sur les conventions à adopter.

Si vous n'avez pas d'opinion précise sur comment votre code doit être, utilisez `black`, si vous n'aimez pas les sorties de `black`, persévérez, on s'habitue.

Si **vraiment** vous avez des goûts sophistiqués, il existe beaucoup d'autre formatteurs: par exemple autopep8, yapf, rope…

`black` et `flake8` sont intégrés à la plupart des éditeurs et IDEs, n'hésitez pas à les utiliser en permanence, vous vous remercierez dans six mois.

## Enfin libres

Ah, c'est bon d'avoir du code qui marche

In [32]:
display(common_neighbours("moi", ancor_t2i, ancor_i2t, ancor_cooc))

['de', 'est', 'euh', 'que', 'je', "c'", 'pas', 'à', 'les', 'et']

Enfin, on a fini, la fonction marche, on peut l'appliquer à ce qu'on veut

In [33]:
display(common_neighbours("bonjour", ancor_t2i, ancor_i2t, ancor_cooc))

['de', 'est', 'euh', 'que', 'je', "c'", 'pas', 'à', 'les', 'et']

In [34]:
display(common_neighbours("Russie", ancor_t2i, ancor_i2t, ancor_cooc))

['de', 'est', 'euh', 'que', 'je', "c'", 'pas', 'à', 'les', 'et']

Euh

In [35]:
display(common_neighbours("Orléans", ancor_t2i, ancor_i2t, ancor_cooc))

['de', 'est', 'euh', 'que', 'je', "c'", 'pas', 'à', 'les', 'et']

In [37]:
display(common_neighbours("médecin", ancor_t2i, ancor_i2t, ancor_cooc))

['de', 'est', 'euh', 'que', 'je', "c'", 'pas', 'à', 'les', 'et']

Il y a **encore** un problème ?

Oui

## Les débuggeurs

### Demo

Aller sur http://pythontutor.com/visualize.html et y coller le code de la factorielle buggué

```python
def factorial(n):
    if n == 0:
        return 1
    res = 1
    for i in range(1, n):
        res = i * res
    return res
    
factorial(9)
```

On peut suivre l'exécution en détails \o/

On appelle ce genre d'interface un *debugger* : on peut suivre l'exécution du programme ligne par ligne, en contrôlant les valeurs des variables et éventuellement en revenant en arrière pour comprendre ce qui se passe

C'est *très* pratique, mais la version de Python Tutor est (volontairement) assez limité. Pour notre code en particulier ça va être compliqué à gérer.

### Pour les grand⋅e⋅s

On va commencer avec [PyCharm](https://www.jetbrains.com/pycharm) qui n'est pas le meilleur IDE pour python mais a à mon avis le meilleur débuggueur en ce moment.

(On regarde le tableau, désolé pour celleux qui ne sont pas là)

### Pour les vraiment très grand⋅e⋅s

Pycharm c'est bien, mais même ça a ses limites

- Quand le code prend toute la RAM
- Quand on débuggue à distance
- Quand on a pas d'interface graphique
- Quand on a pas envie de financer JetBrains

Pour ça il existe une alternative directement incluse dans la bibliothèque standard : [`pdb`](https://docs.python.org/3/library/pdb.html).

Pour le lancer, rien de plus simple (mais ça ne marche pas bien dans un notebook)

```bash
python -m pdb debugme.py
```

Là encore, on regarde le tableau. Pour référence future, tout est dans la doc mais les points importants sont

- Les commandes (il suffit de taper la première lettre
  - `l[ist]`: affiche les lignes autour de l'instruction courante
  - `ll[ist]`: affiche tout la fonction courante
  - `n[ext]`: passer à l'instruction suivante dans la fonction en cours
  - `s[tep]`: passer à l'instruction suivante y compris dans une autre fonction
  - `u[p]`: passer dans le contexte un niveau au dessus (l'instruction qui appelle la fonction courante)
  - `d[own]`: l'inverse de `u[p]`
- Commencer une ligne par un `!` fait exécuter une instruction, bien pratique pour lire le contenu des variables
- Dans n'importe quel progamme, une instruction `breakpoint()` stoppe l'exécution et vous donne une session `pdb`

C'est globalement un peu moins sympa qu'un débuggeur graphique, notamment pour voir en direct les états des variables (mais il y a d'autres alternatives en console) mais ça marche vraiment bien.

## Exercice

- Débugger `debugme.py`, envoyez-moi un pull request sur https://github.com/LoicGrobol/python-im-2 !
- Passer `black` et `flake8` dans vos projets